In [ ]:
artifact_id_dict = {
    # "flowbot": '15ecnnm8',  # run-{}-train_metric_table:v0
    # "traj1": 'wcx2tyyn',
    # "traj5": 'z9japwzr',
    # "traj10": 'dzng8oc9',
    # "traj15": 'ujjttwjx',
    # "traj20": '66vi6gbz',
    "diffuser1": "udh6n9fh",
    "diffuser5": "xbxuwn3s",
    "diffuser10": "qsq7b7d2",
}

In [ ]:
import wandb

api = wandb.Api()
for model in artifact_id_dict.keys():
    artifact_id = artifact_id_dict[model]
    for mode in ["train", "val", "test"]:
        artifact = api.artifact(f'r-pad/open_anything_diffusion/run-{artifact_id}-{mode}_metric_table:v0')
        artifact.download()

In [ ]:
import json
import pandas as pd
neat_dfs = {}
desired_columns_order = {
    "train": ["model", "unweighted_mean", "class_mean", "FoldingChair", "Kettle", "Laptop", "Microwave", "Refrigerator","Stapler", "StorageFurniture", "Switch", "Toilet",	"TrashCan",	"Window"],
    "test": ["model", "unweighted_mean", "class_mean", "Box","Bucket","Dishwasher","Door","KitchenPot", "Oven","Phone","Safe","Table","WashingMachine"],
    "val":["model", "unweighted_mean", "class_mean", "FoldingChair", "Kettle", "Laptop", "Microwave", "Refrigerator","Stapler", "StorageFurniture", "Switch", "Toilet",	"TrashCan",	"Window"],
}
model_order = artifact_id_dict.keys()
for mode in ["train", "val", "test"]:
    neat_dfs[mode] = pd.DataFrame()
    chao_df = pd.DataFrame()
    for model in artifact_id_dict.keys():
        artifact_id = artifact_id_dict[model]
        artifact_local_path = f'/home/yishu/open_anything_diffusion/notebooks/artifacts/run-{artifact_id}-{mode}_metric_table:v0/{mode}_metric_table.table.json'
        with open(artifact_local_path) as file:
            json_dict = json.load(file)
        df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"]).T
        df.columns = df.loc['category']
        df['model'] = model
        df.drop('category', inplace=True)
        df = df.reindex(columns=desired_columns_order[mode])
        chao_df = pd.concat([chao_df, df])
    for metric in ["cos_dist", "rmse", "mag_error"]:
        new_df = chao_df.loc[metric].sort_values(by='model', key=lambda x: x.map({v: i for i, v in enumerate(model_order)}))
        neat_dfs[mode] = pd.concat([neat_dfs[mode], new_df])

In [ ]:
neat_dfs['train'].to_csv('./diffuser_train_metrics.csv')
neat_dfs['train']

In [ ]:
neat_dfs['val'].to_csv('./diffuser_val_metrics.csv')
neat_dfs['val']

In [ ]:
neat_dfs['test'].to_csv('./diffuser_test_metrics.csv')
neat_dfs['test']

In [ ]:
model_order = artifact_id_dict.keys()
# all_dfs['train']['model'] = pd.Categorical(all_dfs['train']['model'], categories=model_order, ordered=True)
metric_df = pd.DataFrame()
for metric in ["cos_dist", "rmse", "mag_error"]:
    new_df = all_dfs['train'].loc[metric].sort_values(by='model', key=lambda x: x.map({v: i for i, v in enumerate(model_order)}))
    metric_df = pd.concat([metric_df, new_df])
metric_df

In [ ]:
all_dfs['val']

In [ ]:
all_dfs['test']